In [2]:
%matplotlib notebook

# Make Bogdanoval et al (2002) Table 4

Generates number of observations (n: column 2) and annual total archived precipitation (P_arch: column 6) columns of Table 4 from Bogdanova et al (2002).  The table summarizes precipitation observations from NP drifting stations from 1956 (NP-5) to 1990 (NP-31).  NP-14 is excluded because Bogdanova et al (2002) consider precipitation from this station to be too high when compared to precipitation observed at other stations.

In [2]:
import sys
sys.path.append('../source')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import os
import glob
import calendar
import re

from merge_npsnow_data import get_precip, get_station_list   #met_filename, get_precip, plot_station_met, merge_one_station

## Load precipitation files and exclude NP 1, 2, 3, 4 and 14

In [12]:
stations = [statid for statid in get_station_list() if statid not in ['01','02','03','04','14']]
df = pd.concat([get_precip(statid) for statid in stations])

## Calculate monthly total precipitation
The dataframe of precipitation amount and type is reorganized by date and station.  Monthly sums are then calculated for each station.  Zero sums are set to NaN.

In [ ]:
# Generate table organized by index time and station
table = pd.pivot_table(df, values='amount', index=df.index, columns='statid')
table = table.resample('MS').sum() # Generate month sums
table = table.where(table > 0) # Set zero to NaN

## Calculate annual total precipitation
Mean monthly precipitation is calculated by averaging precipitation across stations for each month in the record.  Annual total precipitation is calculated by summing all years with 12 monthly average precipitation values.  The number of months used to calculate monthly averages and then annual totals is also given.  Years can have more then 12 monthly averages because more than one station could have been in operation in a given year. 

Comparing the results with values in table 4 gives a good match.  All years, with the exception of 1969, match month counts.  Annual total precipitation is within a few tenths of a millimeter.  1969 stands out as it has one more month of data used in the annual total and annual total precipitation is 2 mm less than in the published table.

In [11]:
month_table = pd.DataFrame({'Parch': table.mean(axis=1), 'n': table.count(axis=1)})
month_table.resample('AS').sum(min_count=12).dropna()

,Parch,n
1956-01-01,158.300000,16.0
1957-01-01,177.650000,20.0
1958-01-01,149.200000,24.0
1962-01-01,121.550000,23.0
1963-01-01,125.850000,24.0
1964-01-01,119.600000,24.0
1965-01-01,104.250000,16.0
1966-01-01,174.700000,20.0
1967-01-01,135.450000,16.0
1969-01-01,190.283333,26.0
